In [1]:
import os
import shutil
import cv2
import imp
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from alexnetG2 import AlexNet
pd.set_option('display.max_rows', 500000)
%matplotlib inline
imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)
current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'images')

# Use SVM file path now instead of FilePathTable.csv
CSV_FilePath = "./svmtrain.csv" # change this for TBT way
num_classes = 2

## Class List
- PathDetails (path)
- ImgDetails  (path)
- Seed_Classifier_St3
- Analyse_PredictionResults

In [2]:
class PathDetails:
    
    def __init__ (self, Path):
        self.PathDF = pd.read_table(Path, sep = ',')  # Converts the csv to DF format
        
    def Get_ImgPath_FilePath(self):
        df = self.PathDF
        fpath = './'+df.ImgListPath_Dir.head(1).values [0] +'/'+df.ImgListPath_File_Name.head(1).values [0]
        return (fpath)
        
    def Get_ChkPoint_FilePath(self,i):
        df = self.PathDF
        fpath = './'+df.Checkpoint_Root_Dir.values [i] +'/' + df.Classifier_Root_Dir.values [i]
        return (fpath)
        
    def Get_ChkPointModel_FilePath(self,i):
        df = self.PathDF
        fpath = './'+df.Checkpoint_Root_Dir.values [i] +'/' + df.Classifier_Root_Dir.values [i] +'/' + df.Classifier_Model_Path.values [i] 
        return (fpath)

    def Get_CaffClassName_FilePath(self,i):
        df = self.PathDF
        fpath = './'+df.Checkpoint_Root_Dir.values [i] +'/' + df.Classifier_Root_Dir.values [i] +'/' + df.CaffeClassNames_File.values [i] 
        return (fpath)    

In [3]:
class ImgDetails:
    
    def __init__ (self,Path):

        img_path_list = []
        imgs = []
        lables = []

        with open(Path, 'r') as img_files:
            # Load all images
            for i, f in enumerate (img_files):
                img_path,lable = f.split(' ')
                img_path_list.append(img_path)
                imgs.append(cv2.imread(img_path))
                lables.append(lable)
        img_files.close
        self.img_path_list = img_path_list
        self.imgs = imgs
        self.lables = lables      

In [4]:
class Seed_Classifier_St3:
    
    def __init__(self):

        self.Initialization_Flag = False
        
    def Initialize_Dataframes(self, path):
        
        self.Path = path
        self.Initialization_Flag = True
        self.Load_DF()
        self.Create_ActiveDF()
        print ("Successful: Initialize_Dataframes")
        
    def Initialization_Check(self):
        
        if not(self.Initialization_Flag): 
            print ("ERROR: Dataframe not initalised. Please run Initialise_Dataframes(<PATH>)")
            return 0
        return 1
       
    def Load_DF (self):
        
        if not (self.Initialization_Check()): return 
        path=self.Path
        self.Loaded_Data = pd.read_table(path, delim_whitespace=True, 
                             names=('Path',
                                    'Set', 
                                    'Prediction', 
                                    'Probability',
                                    'Expectation'),
                             dtype={'Set': np.str, 'Prediction': np.float64, 'Probability': np.float64, 'Expectation': np.float64}) 
        self.Load_DF_Status=True
        print ("Successful: Load_DF")
        
    def  Create_ActiveDF(self):
        
        # Get data
        if not (self.Initialization_Check()): return 
        Loaded_Data=self.Loaded_Data
        
        # Setting Active_df
        Active_df = pd.DataFrame(columns =['Path',
                                    'Category',
                                    'Img_Name',
                                    'Set', 
                                    'ClassifierName',
                                    'Prediction', 
                                    'Probability',
                                    'Expectation'])
        # Active_df +++>
        # Path, Set, Prediction, Probability, Expectation
        Active_df.Path=Loaded_Data.Path
        Active_df.Set=Loaded_Data.Set
        Active_df.Prediction=Loaded_Data.Prediction
        Active_df.Probability=Loaded_Data.Probability
        Active_df.Expectation=Loaded_Data.Expectation

        # Category, Img_Name
        t_path_stack = Loaded_Data.Path.str.split('/', expand=True).stack()
        Active_df.Category = t_path_stack[:, 7]
        Active_df.Img_Name = t_path_stack[:, 8]

        # ClassifierName
        Active_df.ClassifierName ='Classifier' + Loaded_Data.Set 
        self.Active_df = Active_df
        print("Successful: Create_Active_DF")
        
        
    def Create_TProbabilityColLabels(self):
        
        if not (self.Initialization_Check()): return 
        # Create a Confusion matrix for an image
        data = {'Prediction':[1,2,6,7],
                'Classifier1312':np.zeros(4),
                'Classifier2316':np.zeros(4),
                'Classifier3317':np.zeros(4),
                'Classifier4326':np.zeros(4),
                'Classifier5327':np.zeros(4),
                'Classifier6367':np.zeros(4),
               }
        print ("Successful: Create_TProbabilityColLabels")
        return data  
      
    def Create_PredictionResultDfColLabels(self):
        
        if not (self.Initialization_Check()): return 
        # Create a result matrix for an image
        data = {'Class1':np.zeros(1),
                'Class2':np.zeros(1),
                'Class6':np.zeros(1),
                'Class7':np.zeros(1),
                'Prediction':np.zeros(1),
                'Probability':np.zeros(1),
               }
        print ("Successful: Create_PredictionResultDfColLabels")
        return data 
    
    def Get_TotalProbability(self, name):
        
        if not (self.Initialization_Check()): return 
        # Initialise
        iname=name
        df=self.Active_df

        # Create a total probability matrix for an image
        Lables = self.Create_TProbabilityColLabels()
        Probability_Df = pd.DataFrame(Lables)

        # Iteration over the rows of prediction for that image
        Selected_Img_df=self.Search_Img(iname)
        for index, Selected_Row in Selected_Img_df.iterrows():
            Relative_Index =  Probability_Df.index[Probability_Df['Prediction'] == Selected_Row.Prediction]
            Probability_Df.at[Relative_Index,Selected_Row.ClassifierName] =  Selected_Row.Probability
        
        # Obtain the mean from the probs. from the 6 classifiers
        Probability_Df ['Total_Probability'] = Probability_Df.iloc[:, 1:8].mean(axis=1)
        print ("Successful: Get_TotalProbability")
        return Probability_Df
  
    def Get_PredictionResult(self, name):
        
        # Initialization check
        if not (self.Initialization_Check()): return 
        
        # Create a prediction result df for an image
        Probability_Table=self.Get_TotalProbability(name)        
        temp = Probability_Table.loc[Probability_Table['Total_Probability'].idxmax()]
        Lables = self.Create_PredictionResultDfColLabels()
        PredictionResult = pd.DataFrame(Lables)
   
        # Assign Probability
        PredictionResult['Class1']=Probability_Table.iloc[Probability_Table.index[Probability_Table.Prediction==1] [0]]['Total_Probability']
        PredictionResult['Class2']=Probability_Table.iloc[Probability_Table.index[Probability_Table.Prediction==2] [0]]['Total_Probability']
        PredictionResult['Class6']=Probability_Table.iloc[Probability_Table.index[Probability_Table.Prediction==6] [0]]['Total_Probability']
        PredictionResult['Class7']=Probability_Table.iloc[Probability_Table.index[Probability_Table.Prediction==7] [0]]['Total_Probability']
        PredictionResult['Probability']=temp.Total_Probability
        PredictionResult['Prediction']=temp.Prediction
        print ("Successful: Get_PredictionResult")
        return PredictionResult
     
    def Get_UniqueImgList(self):
        """Creates a list of names of images in the dataframe"""
        
        # Initialization check
        if not (self.Initialization_Check()): return 
        
        # Generate List
        Active_df=self.Active_df
        Img_Name_List= list (Active_df['Img_Name'].unique())
        print ("Successful: Get_UniqueImgList")
        return Img_Name_List
              
    def Search_Img(self,name):
        
        # Initialization check
        if not (self.Initialization_Check()): return 
        
        # Using Active_df
        df = self.Active_df
        Selected_Img_df = df[df.Img_Name==name]
        if Selected_Img_df.empty: print('ERROR: DataFrame is empty! \nSuggestion: Please check your image name.')
        return Selected_Img_df
        
    def PrintToFile_Active_df(self,path):
        
        # Initialization check
        if not (self.Initialization_Check()): return 
        
        Active_df=self.Active_df
        os.remove('./Path/Active_df.csv') if os.path.exists('./Path/Active_df.csv') else None
        Active_df.to_csv(r'./Path/Active_df.csv', header=('Path',
                                            'Category',
                                            'Img_Name',
                                            'Set', 
                                            'ClassifierName',
                                            'Prediction', 
                                            'Probability',
                                            'Expectation'), 
                                            index=None, sep=',', mode='a')
        print ("Successful: PrintToFile_Active_df") 

In [5]:
class Analyse_PredictionResults:    

    def __init__(self):

        self.Initialization_Flag=False
        self.Table=pd.DataFrame()
    
    def Extract_ImgInfo(self, ImgDetails, Prediction_Probability):
        df = Prediction_Probability
        df['Path'] = ImgDetails.Path.values [0]
        df['Category'] = ImgDetails.Category.values [0]
        df['Img_Name'] = ImgDetails.Img_Name.values [0]
        df['Expectation'] = ImgDetails.Expectation.values [0]
        df['Match'] = (df['Prediction'] == df['Expectation']) # Match check
        return df
    
    def Append_ToTable(self,df):
        self.Table = self.Table.append(df)
        print ("Successful: Append_ToTable")
        
    def SaveTo_CSV(self):   
        if (self.Table.size.size > 0): 
            self.Table.to_csv('./Result-FPSC3/St3_Prediction_Table.csv',sep = ' ')
            print("Successful: Table was saved.")
        else:
            print("ERROR: Table not save as the Table is empty.")

In [6]:
PathDB = PathDetails(CSV_FilePath) # Separates the contents of the csv. accordingly
ImgList_FilePath = PathDB.Get_ImgPath_FilePath() # Gets the image file path
ImgDB = ImgDetails(ImgList_FilePath) # Loads the images into array from the given path
PathDB.PathDF

,ImgListPath_Dir,ImgListPath_File_Name,Checkpoint_Root_Dir,Classifier_Root_Dir,Classifier_Model_Path,CaffeClassNames_File
0,Path,svmtrain_all6classes.txt,Checkpoints,Checkpoint_1312,model_epoch500.ckpt.meta,caffe_classes_plamseed.py
1,Path,svmtrain_all6classes.txt,Checkpoints,Checkpoint_2316,model_epoch500.ckpt.meta,caffe_classes_plamseed.py
2,Path,svmtrain_all6classes.txt,Checkpoints,Checkpoint_3317,model_epoch500.ckpt.meta,caffe_classes_plamseed.py
3,Path,svmtrain_all6classes.txt,Checkpoints,Checkpoint_4326,model_epoch500.ckpt.meta,caffe_classes_plamseed.py
4,Path,svmtrain_all6classes.txt,Checkpoints,Checkpoint_5327,model_epoch500.ckpt.meta,caffe_classes_plamseed.py
5,Path,svmtrain_all6classes.txt,Checkpoints,Checkpoint_6367,model_epoch500.ckpt.meta,caffe_classes_plamseed.py


## Iteration
Running iterative loop on each of the binary classifier

In [10]:
%%time

# Placeholder for input and dropout rate, x is the input image
x = tf.placeholder(tf.float32, [1, 227, 227, 3])
keep_prob = tf.placeholder(tf.float32)

# Copy all images
ImageArray = ImgDB.imgs

# Set result Directory
rfname = './Result-FPSC3/St_Output_Linux.csv'
os.remove(rfname) if os.path.exists(rfname) else None

i = 0
Model_Path = PathDB.Get_ChkPointModel_FilePath(i)
CheckpointDir_Path = PathDB.Get_ChkPoint_FilePath(i)
sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph(Model_Path)
saver.restore(sess,tf.train.latest_checkpoint(CheckpointDir_Path))
saved_dict = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

model = AlexNet(x, keep_prob, num_classes, [], saved_dict, load_pretrained_weight = True)

# Loop for each checkpoint
for i in range(0, 6): 
    
    # Setting variable values
    original_class_lable = ''
    Model_Path = PathDB.Get_ChkPointModel_FilePath(i)
    CheckpointDir_Path = PathDB.Get_ChkPoint_FilePath(i)
    ClassFile_Path = PathDB.Get_CaffClassName_FilePath(i)
    print(CheckpointDir_Path)
    
    ClassNameModule = imp.load_source('class_names', ClassFile_Path)
    class_names = ClassNameModule.class_names
    saver = tf.train.import_meta_graph(Model_Path)
    saver.restore(sess,tf.train.latest_checkpoint(CheckpointDir_Path))
    saved_dict = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    print('Loading checkpoint = ', i)
    
    # Load the pretrained weights into the model
    model.load_initial_weights(sess)
    score = model.fc8
    prescore = model.fc7

    # Create op to calculate softmax 
    # softmax = tf.nn.softmax(score)

    images = ImgDB.imgs
    labels = ImgDB.lables
    imagefeatures = []
    labelfeatures = []
    scorefeatures = []
    
    # Loop over all images
    for j, image in enumerate(images):
        
        img = cv2.resize(image.astype(np.float32), (227,227))
        img = img - imagenet_mean
        img = img.reshape((1, 227, 227, 3))
        
        # Activation maps output
        layer7Vector = sess.run(prescore, feed_dict={x: img, keep_prob: 1})
        layer8Vector = sess.run(softmax, feed_dict={x: img, keep_prob: 1})
        
        # Obtain feature of the individual image
        imagefeatures.append(layer7Vector) 
        scorefeatures.append(layer8vector)
        
    # Loop over all the labels 
    for a, label in enumerate(labels):
    
        labelfeatures.append(labels)
'''
        # Run the session and calculate the class probability
        probs = sess.run(softmax, feed_dict = {x: img, keep_prob: 1})

        # Get the class name of the class with the highest probability
        class_name = class_names[np.argmax(probs)]

        # Lables converted to the respective labels of the classes
        original_class_lable = class_name[5]
    
        # Filename to write
        with open(rfname, 'a') as myfile:
            ptr = myfile.write(ImgDB.img_path_list[j]+" "+CheckpointDir_Path[-4:]+" "+class_name[5]+" " +"%.4f" %probs[0,np.argmax(probs)]+' '+ImgDB.lables[j]) 
'''

INFO:tensorflow:Restoring parameters from ./Checkpoints/Checkpoint_1312/model_epoch500.ckpt
./Checkpoints/Checkpoint_1312
INFO:tensorflow:Restoring parameters from ./Checkpoints/Checkpoint_1312/model_epoch500.ckpt
Loading checkpoint =  0
Loading Pretrained weight.
./Checkpoints/Checkpoint_2316
INFO:tensorflow:Restoring parameters from ./Checkpoints/Checkpoint_2316/model_epoch500.ckpt
Loading checkpoint =  1
Loading Pretrained weight.
./Checkpoints/Checkpoint_3317
INFO:tensorflow:Restoring parameters from ./Checkpoints/Checkpoint_3317/model_epoch500.ckpt
Loading checkpoint =  2
Loading Pretrained weight.
./Checkpoints/Checkpoint_4326
INFO:tensorflow:Restoring parameters from ./Checkpoints/Checkpoint_4326/model_epoch500.ckpt
Loading checkpoint =  3
Loading Pretrained weight.
./Checkpoints/Checkpoint_5327
INFO:tensorflow:Restoring parameters from ./Checkpoints/Checkpoint_5327/model_epoch500.ckpt
Loading checkpoint =  4
Loading Pretrained weight.
./Checkpoints/Checkpoint_6367
INFO:tensorfl

In [ ]:
'''
# Initilize Dataframe
# Parameter: none
newresult = './finalpath_rabbi.csv'
myDF = Seed_Classifier_St3()
_ = myDF.Initialize_Dataframes(newresult)

# List of images 
# Parameter: myDF
myImg_Name_List = myDF.Get_UniqueImgList()

# Creating a DF to analyse the result
Analyse_ResultDF=Analyse_PredictionResults()

for iname in myImg_Name_List:
    
    # Get the max probability for prediction
    PredictionResult_DF = myDF.Get_PredictionResult(iname)

    # Get current image details
    ImageDetails_DF = myDF.Search_Img(iname)
    
    # Converting the dataframe into a suitable format
    # so that it can be added as row
    Row = Analyse_ResultDF.Extract_ImgInfo(ImageDetails_DF,PredictionResult_DF)
    _ = Analyse_ResultDF.Append_ToTable(Row)
    
# Save result
_ = Analyse_ResultDF.SaveTo_CSV()
#Analyse_ResultDF.Table
'''

## Flatten the feature mapping

flatimagefeatures contains all the features of all the training images of total size (371,4096) where 371 is the total number of training images and 4096 is the feature maps of each image. To input to SVM, squeeze() is use to change it a 1D vector. Same is done to the labels

In [211]:
# Flatten into 1D vector

flatimagefeatures = np.squeeze(imagefeatures)
flatlabelfeatures = np.array(labelfeatures, dtype = int) 
flatlabelfeatures = ((flatlabelfeatures[0:1]).T).ravel() 
flatlabelfeatures = flatlabelfeatures.ravel()
flatlabelfeatures.shape
flatimagefeatures.shape

(371,)

(371, 4096)

In [209]:
# Save feature vector if needed

np.savetxt("featuresReLu.csv", flatimagefeatures, delimiter=",", fmt='%.3e')

## SVM

In [1]:
%%time
# Training the linear SVM using SVC
import sklearn
import sklearn.svm
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

modelSVM = LinearSVC(random_state=0, loss='hinge')
modelSVM.fit(flatimagefeatures, flatlabelfeatures)

NameError: name 'flatimagefeatures' is not defined

## Testing images and labels for SVM

In [2]:
# Predict the class of a new image using our trained model

CSV_FilePath_test = "./testingSVM.csv" # Test file path
PathDB_test = PathDetails(CSV_FilePath_test)
ImgList_FilePath_test = PathDB_test.Get_ImgPath_FilePath()
ImgDB_test = ImgDetails(ImgList_FilePath_test)

imagefeaturesTest = []
labelfeaturesTest = []

# Train all image, then test with something else

for jTest, imageTest in enumerate(ImgDB_test.imgs):

    img = cv2.resize(imageTest.astype(np.float32), (227,227))   
    img = img - imagenet_mean
    img = img.reshape((1, 227, 227, 3))
    img = np.array(img).astype(np.float32)
    layer7VectorTest = sess.run(prescore, feed_dict={x: img, keep_prob: 1}) # Obtain feature of the individual image
    imagefeaturesTest.append(layer7VectorTest)# append all the features
    
for aTest, labelTest in enumerate(ImgDB_test.lables):
    
    labelfeaturesTest.append(ImgDB_test.lables)
    
# Flatten into 1D vector

flatimagefeaturesTest = np.squeeze(imagefeaturesTest)  
flatlabelfeaturesTest = np.array(labelfeaturesTest, dtype = int)
flatlabelfeaturesTest = (flatlabelfeaturesTest[0:1]).T

NameError: name 'PathDetails' is not defined

## Creating the predictions with the test set for SVM

In [188]:
# Create prediction with the test set

predicted_class_testarray = []

for i in range(len(flatlabelfeaturesTest)):
    f2test = flatimagefeaturesTest[i, :] # Getting the test image features up to 370 image and predicting
    predicted_class_test = modelSVM.predict([f2test]) # Prediction of the image using the features learned
    predicted_class_testarray.append(predicted_class_test)  

## Plotting function for confusion matrix

In [214]:
import itertools
def plot_confusion_matrix(cm, target_names, title = 'Confusion Matrix', cmap = None, normalize = True):
    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize = (7, 7))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)

    if target_names is not None:
        target_names = ['Class 1', 'Class 2', 'Class 6', 'Class 7'] # Use string names
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=0)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.3f}".format(cm[i, j]),horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('REAL')
    plt.xlabel('PREDICTED\n Accuracy = {:0.3f} || Error = {:0.3f}'.format(accuracy, misclass))
    plt.tight_layout()
    plt.show()

## Confusion matrix

In [1]:
# Checking the prediction against real ones

PredSVM = (np.array(predicted_class_testarray)).ravel()
TrueSVM = flatlabelfeaturesTest.ravel()
cm = confusion_matrix(flatlabelfeaturesTest.ravel(), predicted_class_testarray, labels = [1, 2, 6, 7]) # 4 classes
plot_confusion_matrix(cm, [1, 2, 6, 7], normalize=False) # Normalize to get probs. 
print('SVM Accuracy =', np.mean(PredSVM == TrueSVM) * 100.0)

NameError: name 'np' is not defined